In [10]:
import pymongo
client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
db = client.rawData
db.collection_names()
collection = client.rawData.ipeenInfo



In [ ]:
import json
import pymongo
from pymongo import UpdateOne
import numpy
import math
from math import radians, cos, sin, asin, sqrt
from collections import Counter
import time
import sys
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
import smtplib
import gc
gc.enable() 



def haversine(lng1, lat1, lng2, lat2):
    # 将十进制度数转化为弧度
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    # haversine公式
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlng / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r * 1000


b=time.time()
def mailTo(title,mailAdds,message,whoSend='taiwanInfoSuFood'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()  

mailTo(title="taiwanInfoSuFoodbegin",mailAdds=["andy.yuan@wowprime.com"],message="分析開始")
try:
    ######################################################第一次跑用stoneTwo來篩選沒超商的點
    b=time.time()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collectionTaiwan = client.rawData.taiwanInfoMaBoss
#     collectionTaiwan = client.rawData.taiwanInfoSuFood
    ##################################################第一次跑不執行
    TWLocs=list(collectionTaiwan.find({}, {'_id': False}))
    ##################################################第一次跑不執行
    
    ######################################################IPEEN
    # collection = client.rawData.wowprimeipeen
    collection = client.rawData.ipeenInfo
    ipeendata = list(collection.find({}, {'_id': False}))
    ipeendata = [dien for dien in ipeendata if dien['status'] == "正常營業"
                 and dien['lat'] > 18
                 and dien['lat'] < 27
                 and dien['lng'] < 125
                 and dien['lng'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0
                 and dien['averagecost'] < 250]

    ######################################################IPEEN
    ######################################################conStore0320
    collection = client.rawData.conStore
    conStoreData = list(collection.find({}, {'_id': False}))
    ######################################################conStore0320
    ######################################################Watsons
    collection = client.rawData.Watsons
    watsonsData = list(collection.find({}, {'_id': False}))
    ######################################################Watsons
    ######################################################canSuMe
    collection = client.rawData.canSuMe
    canSuMeData = list(collection.find({}, {'_id': False}))
    ######################################################canSuMe
#     ######################################################carrefour0504
    collection = client.rawData.MRTinfo
    MRTData = list(collection.find({}, {'_id': False}))
#     ######################################################carrefour0504
    ######################################################pxmart0504
    collection = client.rawData.eslite
    esliteData = list(collection.find({}, {'_id': False}))
    ######################################################pxmart0504
#     ######################################################Clinic0531
#     collection = client.rawData.infoClinic
#     clinicData = list(collection.find({}, {'_id': False}))
#     ######################################################Clinic0531
#     ######################################################Tstore0531
#     collection = client.rawData.info3Store
#     tStoreData = list(collection.find({}, {'_id': False}))
#     ######################################################Tstore0531
    ##################################################第一次跑先去除latlng以外的資訊
    TWLocs = [{"lat":i["lat"],"lng":i["lng"]} for i in TWLocs if ('NconStore_Analyze' in i and
              i['NconStore_Analyze'] >0)]
#     TWLocs = [i for i in TWLocs if ('NconStore_Analyze' in i and
#               i['NconStore_Analyze'] >0)]
    ##################################################第一次跑先去除latlng以外的資訊
    allN=len(TWLocs)
    print("有{}筆資料要分析".format(allN))
    radius=500
    n = 0

    for TWLoc in TWLocs:
        score=0
        n += 1
        if n % 50 == 0:
            e=time.time()
            print("已完成{}%分析-{}項,花費{}秒".format(round(n/allN,4)*100,n,round(e-b)))
        if n % 2000 == 0:
            gc.collect()
            client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
            collection = client.rawData.taiwanInfoSuFood
            
            ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            datas=[i for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,datas)]
            try:
                collection.bulk_write(operations ,ordered=False)
            except Exception as er:
                print(er)
            client.close()

        TWLoc["areaRadius_Analyze"] = radius
        #############################################################0523Score
        #+2
        TWLoc["NconStore_Analyze"] = len([dien['name'] for dien in conStoreData if "lng" in dien and haversine(lng1=float(dien["lng"]),
                                                                                lat1=float(dien["lat"]),
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= radius])
            
        #+2
        TWLoc["Nwatson_Analyze"] = len([dien for dien in watsonsData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        TWLoc["NcanSuMe_Analyze"] = len([dien for dien in canSuMeData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        
        #+2
        TWLoc["NMRT_Analyze"] = len([dien for dien in MRTData
                                  if haversine(lng1=float(dien["lng"]),
                                               lat1=float(dien["lat"]),
                                               lng2=TWLoc["lng"],
                                               lat2=TWLoc["lat"]) <= radius])
        #+2
        TWLoc["Neslite_Analyze"] = len([dien for dien in esliteData
                                  if haversine(lng1=float(dien["lng"]),
                                               lat1=float(dien["lat"]),
                                               lng2=TWLoc["lng"],
                                               lat2=TWLoc["lat"]) <= radius])

        rangeCost = 1.4
        
        #+2
        goodDien = [dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['averagecost'] > 399]
        
        TWLoc["NgoodDien"] = len(goodDien)
        
        #-1
        Veget = len([dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['bigstyle']=='素食'
                   and dien['smallstyle']=='西式'
                   and dien['averagecost'] < 450
                   and dien['averagecost'] > 200])
        TWLoc["NVeget_analyze"] = Veget
        

        #############################################################0523Score
#     for i in TWLocs:
#         score=0

#         if i['NsimCostDien']>=17:
#             score+=3
#         elif i['NsimCostDien']>=9:
#             score+=2
#         elif i['NsimCostDien']>=5:
#             score+=1

#         if i['NTea_analyze']>=11:
#             score+=2
#         elif i['NTea_analyze']>=6:
#             score+=1

#         if i['NThreeMom_analyze']+i["Nmc_Analyze"]+i["Nken_Analyze"]>=1:
#             score+=1


#         if i['NconStore_Analyze']>=13:
#             score+=2
#         elif i['NconStore_Analyze']>=5:
#             score+=1

#         if i['Npxmart_Analyze']+i['Nwatson_Analyze']>=3:
#             score+=2
#         elif i['Npxmart_Analyze']+i['Nwatson_Analyze']>=1:
#             score+=1

#         i['score']=score
        
    
    e = time.time()


#     client.close()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collection = client.rawData.taiwanInfoSuFood
    ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs]
    operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
    try:
        collection.bulk_write(operations ,ordered=False)
    except Exception as er:
        print(er)
    client.close()
    
    e = time.time()
    print(e-b)
    
    
    mailTo(title="taiwanInfoSuFoodSuccess",mailAdds=["andy.yuan@wowprime.com"],message="分析完畢_耗時{}秒".format(e-b))
except Exception as er:
    errorMsg=str(er)+"出錯位置:"+str(sys.exc_info()[2].tb_lineno)
    mailTo(title="taiwanInfoSuFoodFail",mailAdds=["andy.yuan@wowprime.com"],message=errorMsg)

有50671筆資料要分析
已完成0.1%分析-50項,花費43秒
已完成0.2%分析-100項,花費81秒
已完成0.3%分析-150項,花費119秒
已完成0.38999999999999996%分析-200項,花費157秒
已完成0.49%分析-250項,花費194秒
已完成0.59%分析-300項,花費231秒
已完成0.69%分析-350項,花費269秒
已完成0.79%分析-400項,花費307秒
已完成0.89%分析-450項,花費344秒
已完成0.9900000000000001%分析-500項,花費382秒
已完成1.09%分析-550項,花費419秒
已完成1.18%分析-600項,花費457秒
已完成1.28%分析-650項,花費494秒
已完成1.38%分析-700項,花費532秒
已完成1.48%分析-750項,花費570秒
已完成1.58%分析-800項,花費609秒
已完成1.68%分析-850項,花費647秒
已完成1.78%分析-900項,花費685秒
已完成1.87%分析-950項,花費723秒
已完成1.97%分析-1000項,花費762秒
已完成2.07%分析-1050項,花費800秒
已完成2.17%分析-1100項,花費839秒
已完成2.27%分析-1150項,花費877秒
已完成2.37%分析-1200項,花費914秒
已完成2.4699999999999998%分析-1250項,花費952秒
已完成2.5700000000000003%分析-1300項,花費989秒
已完成2.6599999999999997%分析-1350項,花費1027秒
已完成2.76%分析-1400項,花費1065秒
已完成2.86%分析-1450項,花費1103秒
已完成2.96%分析-1500項,花費1141秒
已完成3.06%分析-1550項,花費1179秒
已完成3.16%分析-1600項,花費1217秒
已完成3.26%分析-1650項,花費1255秒
已完成3.35%分析-1700項,花費1292秒
已完成3.45%分析-1750項,花費1330秒
已完成3.55%分析-1800項,花費1368秒
已完成3.65%分析-1850項,花費1406秒
已完成3.75%分析-1900項,花費1444秒
已完成3.85%分析-1950項,

已完成28.22%分析-14300項,花費10976秒
已完成28.32%分析-14350項,花費11015秒
已完成28.42%分析-14400項,花費11054秒
已完成28.52%分析-14450項,花費11092秒
已完成28.62%分析-14500項,花費11133秒
已完成28.71%分析-14550項,花費11172秒
已完成28.810000000000002%分析-14600項,花費11213秒
已完成28.910000000000004%分析-14650項,花費11254秒
已完成29.01%分析-14700項,花費11295秒
已完成29.110000000000003%分析-14750項,花費11339秒
已完成29.21%分析-14800項,花費11381秒
已完成29.310000000000002%分析-14850項,花費11420秒
已完成29.409999999999997%分析-14900項,花費11458秒
已完成29.5%分析-14950項,花費11500秒
已完成29.599999999999998%分析-15000項,花費11542秒
已完成29.7%分析-15050項,花費11583秒
已完成29.799999999999997%分析-15100項,花費11622秒
已完成29.9%分析-15150項,花費11660秒
已完成30.0%分析-15200項,花費11702秒
已完成30.099999999999998%分析-15250項,花費11745秒
已完成30.19%分析-15300項,花費11788秒
已完成30.29%分析-15350項,花費11830秒
已完成30.39%分析-15400項,花費11872秒
已完成30.490000000000002%分析-15450項,花費11914秒
已完成30.59%分析-15500項,花費11955秒
已完成30.69%分析-15550項,花費11999秒
已完成30.79%分析-15600項,花費12038秒
已完成30.89%分析-15650項,花費12080秒
已完成30.98%分析-15700項,花費12123秒
已完成31.080000000000002%分析-15750項,花費12163秒
已完成31.180000000000003%分析-15800項,花費